## Imports

In [28]:
from transformers import BertTokenizer

# from torch.utils.data import Dataset
from models.CustomDatasetBinary import CustomDatasetBinary
from torch.utils.data import DataLoader

from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Datasets & loaders

In [52]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df = pd.read_csv('../data/MD_LoD.csv', index_col = 0)
df

# exclude labels == 2 (slightly agreement)
df = df[df['disagreement_lev'] != 2]

train_set = CustomDatasetBinary(df[df['split'] == 'train'], tokenizer, label_col = 'disagreement_lev')
val_set = CustomDatasetBinary(df[df['split'] == 'dev'], tokenizer, label_col = 'disagreement_lev')
test_set = CustomDatasetBinary(df[df['split'] == 'test'], tokenizer, label_col = 'disagreement_lev')

print(f'len: \n\tTrain: {len(train_set)}\n\tVal: {len(val_set)}\n\tTest: {len(test_set)}')

train_loader = DataLoader(train_set, batch_size = 8, shuffle=True)
val_loader = DataLoader(val_set, batch_size = 8, shuffle=True)
test_loader = DataLoader(test_set, batch_size = 8, shuffle=True)

len: 
	Train: 4662
	Val: 787
	Test: 2148


## Model

In [69]:
from models.TransformerBinaryClassifier import TransformerBinaryClassifier

from transformers import BertModel
bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [70]:
model = TransformerBinaryClassifier(
    model = bert_model,
    dropout_rate = .2,
)

In [71]:
inp, lbl = next(iter(train_loader))
model(
    inp['input_ids'],
    inp['attention_mask'],
)

tensor([[0.0000, 0.3997],
        [0.0000, 0.2425],
        [0.0000, 0.0791],
        [0.0000, 0.4047],
        [0.0000, 0.1818],
        [0.0000, 0.2804],
        [0.0000, 0.2164],
        [0.0000, 0.5631]], grad_fn=<ReluBackward0>)